# Environment setup

In [ ]:
from scipy.io import loadmat
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
np.random.seed(260)
import itertools

# Path to musicImagery dataset
dataDir = r'../../data/musicImagery/'
#dataDir = r'D:\marion_music_imagery\datasetCND_musicImagery\musicImagery'

# Load stimuli
Note: ```stimIndxs``` range from 1 to 4 and ```condIdxs``` range from 1-2.
These are _not_ 0-indexed.

In [ ]:
stim_mat = loadmat(dataDir+r"/dataCND/dataStim.mat", simplify_cells = True) 
stim = stim_mat['stim']
# print(stim)

stimIdxs = stim['stimIdxs']
print(f'stimIdxs shape (N trials): {stimIdxs.shape}')

condIdxs = stim['condIdxs']
print(f'condIdxs shape (N trials): {condIdxs.shape}')

condNames = stim['condNames']
print(f'condNames shape (P conditions): {condNames.shape}')

stim_events = stim['data']
print(f'events shape (M features, N trials): {stim_events.shape}')

stim_event_labels = stim['names']
print(f'event names shape (M features): {stim_event_labels.shape}')

fs_stim = stim['fs'] # ignore, use fs from data struct (64 Hz)
print(f'fs: {fs_stim} Hz')

In [ ]:
# Handy mapping of indices to labels
stimId_to_Song_map = {
    2: 'chor-038', 
    1: 'chor-096', 
    3: 'chor-101',
    4: 'chor-019', 
} # from read_stim_midi.ipynb

stimLabel_to_id_map = {
    'chor-038': 2, 
    'chor-096': 1, 
    'chor-101': 3,
    'chor-019': 4, 
} # reverse mapping

stimLabel_to_sheet_order_map = {
    'chor-038': 1, 
    'chor-096': 2, 
    'chor-101': 3,
    'chor-019': 4, 
} # order of songs on sheet music

condId_to_State_map = {
    1: 'Listening', 
    2: 'Imagery'
} # from condNames

# Stim Channel

Finding tactile metronome onsets

In [ ]:
# From Guilhem's note
barOnsets = [1, 157] # in sample indices

cueTime = 310 # third bar
for barIdx in range(10): # 12 bars total
    barOnsets.append(cueTime)
    if barIdx%2 == 1:
        cueTime += 153
    else:
        cueTime += 154
        
cueTime -= 1 # index from 0

# Make a new exp vector for metronome hits
cueExp = np.zeros_like(stim_events[1][0]) # same size as the first expectation vector
cueExp[barOnsets] = 1

plt.figure()
plt.stem(cueExp, '*')
plt.xlabel('Time [sample]')
plt.title('Tactle Metronome Onsets')
plt.show()

Note onsets for stim channel

In [ ]:
stim = {}
for stimLabel in [
    'chor-038', 
    'chor-096', 
    'chor-101',
    'chor-019']: # top to bottom on the sheet music

    stim[stimLabel] = {}
    
    # Collect note and beat onsets for current stim
    stimIdx = stimLabel_to_id_map[stimLabel]
    stimExp = stim_events[1][11*(stimIdx-1)] # expectations vector for current stim
    note_onsets = stimExp > 0
    stim[stimLabel]['notes'] = note_onsets
    stim[stimLabel]['beats'] = cueExp

These indices will be used later to identify song and condition for a given subject's trial order:

In [ ]:
# Collect stim idxs by condition and song
idxs = {} # dict to store idxs for each condition and stim/song idx

for condIdx in np.unique(condIdxs): # two conditions
    condName = condNames[condIdx-1] # MATLAB indexes from 1...
    idxs[condName] = {}
    
    for stimIdx in np.unique(stimIdxs):
        
        stimName = stimId_to_Song_map[stimIdx]
        
        print(f'{condName}: Song {stimName}: Indices:') 
        matched_ndx = np.where(
            (stimIdxs == stimIdx) & (condIdxs == condIdx)
            )[0]
        print(matched_ndx)
        
        idxs[condName][stimName] = matched_ndx

# Load EEG data for all subjects

Load pre-processed blink data for multiple subjects

In [ ]:
eog_peaks_filename = '../../data/eog_peaks/eog_peaks_21subs.npy'
eog_peaks = np.load(eog_peaks_filename, allow_pickle=True)
eog_peaks = eog_peaks.flat[0]
eog_peaks[1].keys()

In [ ]:
eog_peaks_merged_trials = {}
fs_Hz = 64.0

for subjectID in range(1, 21+1):
    npy_path = f'../../data/eog_peaks/merged_raws/eog_peaks_sub{subjectID}_merged_raws.npy'
    arr = np.load(npy_path, allow_pickle=True)
    arr = arr.flat[0]
    eog_peaks_merged_trials[subjectID] = arr[subjectID]

Replace old eog_peak blinks with eog_peaks_merged_trials:

In [ ]:
# Get blinks by trial
def getTrialBlinksAndSource(subjectID, trial, eog_peaks_merged_trials, n_trial = 1803):
    n_trial_start = trial * n_trial
    n_trial_end = (trial + 1) * n_trial
    
    ica_eog_events = eog_peaks_merged_trials[subjectID]['ica_eog_events']
    #print(ica_eog_events)
    
    trial_blinks = ica_eog_events[
        (n_trial_start <= ica_eog_events) & (ica_eog_events < n_trial_end)
    ]
    trial_blinks -= n_trial_start # reset trial start to t=0

    source = eog_peaks_merged_trials[subjectID]['source']
    trial_source = source[n_trial_start:n_trial_end]

    return trial_blinks, trial_source


In [ ]:
PLOT_SOURCE = True
OVERWRITE_EOG_PEAKS = True

for subjectID in eog_peaks:
    if PLOT_SOURCE:
        fig, ax = plt.subplots(figsize = (24, 32))

    trial = 0
    deltas = []
    for stimIdx in eog_peaks[subjectID]:
        ica_eog_events_old = eog_peaks[subjectID][stimIdx]['ica_eog_events']
        source_old = np.copy(eog_peaks[subjectID][stimIdx]['source']) # to prevent modifying in place

        ica_eog_events_new, source_new = getTrialBlinksAndSource(
            subjectID, 
            trial, 
            eog_peaks_merged_trials, 
            n_trial = 1803,
        )
        source_new = np.copy(source_new)
        
        num_blinks_old = len(ica_eog_events_old)
        num_blinks_new = len(ica_eog_events_new)
        delta = num_blinks_new - num_blinks_old
        deltas.append(delta)

        if delta != 0:
            summary = f'sub{subjectID}\ttrial{trial}\tstimIdx{stimIdx}'
            summary += f'\told: {num_blinks_old}\tnew: {num_blinks_new}\tdel: {delta}'
            print(summary)

        if PLOT_SOURCE:
            source_old /= np.max(np.abs(source_old))
            source_new /= np.max(np.abs(source_new))

            line, = ax.plot(
                trial - source_old, 
                label='old',
                c = 'k',
                alpha = 0.2, 
                #ls = ':',
            )
            new_line, = ax.plot(
                trial + source_new, 
                label='new',
                #c = line.get_color(),
            )
            ax.plot(
                ica_eog_events_new,
                trial + source_new[ica_eog_events_new], 
                '*',
                label='new',
                c = new_line.get_color(),
            )

        if OVERWRITE_EOG_PEAKS:
            eog_peaks[subjectID][stimIdx]['ica_eog_events'] = ica_eog_events_new
            eog_peaks[subjectID][stimIdx]['source'] = source_new
            
        trial += 1

    if PLOT_SOURCE:
        plt.ylabel('Trial #')
        plt.xlabel('Sample #')
        plt.title(f'Subject: {subjectID}\nDeltas: {deltas}')
        plt.show()

Plot top IC (highest EOG score) for all subjects, conditions, songs, and trials

In [ ]:
AVG_SCALE = 10
AVG_OFFSET = 15

if 0:
    for subjectID in eog_peaks:

        # Make a new figure per subject, song, and condition 
        plt.figure(figsize=(16, 16))

        t = np.arange(start=0, stop=1803, step=1, dtype='float64')

        # Container for collecting average across trials
        src_avg = [np.zeros_like(t)]*8
        N_avg = 11 #len(eog_peaks[subjectID])
        
        trialOffset = [0]*8

        # Plot eog_source traces for each subject, trial
        for trialIdx in eog_peaks[subjectID]:

            eog_source = eog_peaks[subjectID][trialIdx]['source']
            max_peak = np.max(eog_source)
            min_peak = np.min(eog_source)
            if np.abs(min_peak) > np.abs(max_peak):
                eog_source /= min_peak # normalize, flip polarity
            else:
                eog_source /= max_peak # normalize

            stimIdx = stimIdxs[trialIdx]
            condIdx = condIdxs[trialIdx]

            pltIdx = 4*(condIdx-1) + stimIdx

            # Plot Fp1 data for these trials
            plt.subplot(4, 2, pltIdx)

            # Plot and add an offset for each trial
            plt.plot(eog_source + trialOffset[pltIdx-1]) # chan0 is FP1

            # Average
            src_avg[pltIdx-1] += eog_source / N_avg

            trialOffset[pltIdx-1] += 1
                
        # Also plot notes/beats and add titles
        for stimIdx in [1, 2, 3, 4]: # ordering of stim doesn't matter for plotIdx
            stimLabel = stimId_to_Song_map[stimIdx]
            notes = t[stim[stimLabel]['notes']]
            beats = t[stim[stimLabel]['beats']>0]

            for condIdx in [1, 2]:
                condLabel = condId_to_State_map[condIdx]

                pltIdx = 4*(condIdx-1) + stimIdx

                # Plot Fp1 data for these trials
                plt.subplot(4, 2, pltIdx)

                # Plot the average
                plt.plot(AVG_SCALE * src_avg[pltIdx-1] + AVG_OFFSET, 
                        label=f'{AVG_SCALE} x avg')

                # draw beat/note onsets
                for note in notes:
                    plt.axvline(x=note, c='r', linewidth=1, alpha=0.5, linestyle=':')
                for beat in beats:
                    plt.axvline(x=beat, c='r', linewidth=1, alpha=0.9) #0 * t_bar[beats_bar], '*')

                title_str = '' #'='*50 + '\n'
                title_str += f'Subject: {subjectID}\n'
                title_str += f'{condLabel}: {stimLabel}\n'
                title_str += f'Trials: {N_avg}'
                plt.title(title_str)

                plt.xlabel('Time [Samples]')
                plt.ylabel('Trial #')
                plt.legend(loc='upper right', prop={'size': 6}) #, bbox_to_anchor=[1.5, 0.5])
                #print(title_str)

                pltIdx += 1

        plt.tight_layout()

In [ ]:
from scipy import signal

fs = 64
two_pi = 2*np.pi

lo_cutoff_Hz = 1
hi_cutoff_Hz = 8

sos = signal.butter(
    N = 2,
    Wn = [lo_cutoff_Hz, hi_cutoff_Hz],
    btype = 'bandpass',
    analog = False,
    output = 'sos',
    fs = fs,
)

'''
b, a = signal.butter(
    N = 2,
    Wn = [0.1, 30],
    btype = 'bandpass',
    analog = False,
    output = 'ba',
    fs = 64,
)
'''

# Freq response 
# code from https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html
#ω, h = signal.freqs(b, a)

plt.figure()

ω, h = signal.sosfreqz(sos)
plt.semilogx(ω/two_pi, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:
# Try different BPF settings to try and improve EOG event detection 

def ica_find_eog_events_custom(raw_trials, subjectID, trialIdx, source, l_freq, h_freq):
    
    #print(f'[{trialIdx}] from {raw_trials[subjectID]}')
    
    raw = raw_trials[subjectID][trialIdx-1]

    # Find EOG events frop top IC
    ica_eog_event_id = 3
    ica_eog_events = mne.preprocessing.ica_find_eog_events(
        raw,
        eog_source = source,
        event_id = ica_eog_event_id,
        l_freq = l_freq,
        h_freq = h_freq,
        verbose='ERROR',
        #ch_name = ['Fpz', 'Fp1', 'Fp2', 'AF7', 'AF3', 'Afz', 'AF4', 'AF8'],
    )

    return ica_eog_events

from scipy.signal import find_peaks

def find_peaks_from_ica_source(
    eog_source,
    fs = 64,
    height = 0.5,
    distance = 32,
    bpf = True,
    l_freq = 1,
    h_freq = 10,
    order = 2,
):
    # Normalize eog source
    max_peak = np.max(eog_source)
    min_peak = np.min(eog_source)
    if np.abs(min_peak) > np.abs(max_peak):
        eog_source /= min_peak # normalize, flip polarity
    else:
        eog_source /= max_peak # normalize

    if bpf:
        sos = signal.butter(
            N = order,
            Wn = [l_freq, h_freq],
            btype = 'bandpass',
            analog = False,
            output = 'sos',
            fs = fs,
        )
        eog_source = signal.sosfiltfilt(sos, eog_source)

    # Find peaks
    ica_eog_events, _ = find_peaks(
        eog_source,
        height = height, 
        distance = distance, # could miss double blinks 
    )

    return ica_eog_events

# Run peak-finder on top EOG IC
TODO: see if this improves blink detection?

In [ ]:
LOAD_FILTERED_BLINKS = False

if LOAD_FILTERED_BLINKS:
    eog_peaks = np.load('eog_peaks_21subs_bpf.npy', allow_pickle=True)
    eog_peaks = eog_peaks.flat[0]

else:    
    CUSTOM_BPF = False
    USE_PEAK_FINDER = True
    UPDATE_EOG_PEAKS = True

    # Plot and add an offset for each trial
    t = np.arange(start=0, stop=1803, step=1)

    for subjectID in eog_peaks:
        
        plt.figure(figsize=(8, 48))
        pltIdx = 1

        # Path to data for selected subject
        sub1_mat = loadmat(dataDir+f"/dataCND/dataSub{subjectID}.mat", simplify_cells = True) 

        # Randomized trial order for current subject
        orig_trial_pos = sub1_mat['eeg']['origTrialPosition']
        #print(f'orig_trial_pos shape: {orig_trial_pos.shape}')

        # Get current subject's song and condition ordering (since its randomized)
        thisSubStimOrder = stimIdxs[(orig_trial_pos-1)] # convert to being zero indexed
        thisSubCondOrder = condIdxs[(orig_trial_pos-1)]
        #print(f'Subject{subjectID} stim order: {thisSubStimOrder}')
        #print(f'Subject{subjectID} cond order: {thisSubCondOrder}')
        

        for stimIdx in [1, 2, 3, 4]: # top to bottom on the sheet music
            stimLabel = stimId_to_Song_map[stimIdx]

            for condIdx in [1, 2]:
                condLabel = condId_to_State_map[condIdx]

                # Find trials matching this song and condition
                trialsToEpoch = np.where(
                    (thisSubStimOrder == stimIdx) & (thisSubCondOrder == condIdx)
                )[0]

                trials_to_plot = trialsToEpoch # idxs[condLabel][stimLabel]

                # Plot Fp1 data for these trials
                plt.subplot(8, 1, pltIdx)
                offset = 0
                N_avg = 11
                source_avg = np.zeros_like(t, dtype='float64')
                
                for trialIdx in trials_to_plot:
                    source = eog_peaks[subjectID][trialIdx]['source']

                    if CUSTOM_BPF == True:
                        ica_eog_events = ica_find_eog_events_custom(
                            raw_trials, 
                            subjectID, 
                            trialIdx, 
                            source,
                            l_freq = 1,
                            h_freq = 3,
                        )
                    elif USE_PEAK_FINDER:
                        ica_eog_events = find_peaks_from_ica_source(
                            source,    
                            height = 0.25,
                            distance = 20, # assuming half_width = 10 below
                            l_freq = 1,
                            h_freq = 8,
                            order = 2,
                        )
                    else:
                        ica_eog_events = eog_peaks[subjectID][trialIdx]['ica_eog_events']

                    if UPDATE_EOG_PEAKS:
                        eog_peaks[subjectID][trialIdx]['ica_eog_events'] = ica_eog_events

                    # print(t)

                    # plt.plot(fp1_data_hp + offset -1)
                    ic_trace, = plt.plot(
                        t,
                        source + offset,
                        linewidth = 0.5,
                    )
                    
                    plt.plot(
                        t[ica_eog_events], 
                        source[ica_eog_events] + offset, 
                        '*',
                        color = ic_trace.get_color())

                    offset += 1 # increment offset for next run

                    # Average
                    source_avg += source / N_avg

                # Plot the average
                # print(fp1_avg)
                offset = 12
                plt.plot(10 * source_avg + offset, label='scaled avg')

                # Plot the note onsets
                plt.plot(offset * note_onsets, 'r', 
                        label='note', 
                        alpha=0.2,
                        linewidth=1
                        ) # 1 = expectations

                # Plot metronome hits (could overlap with some notes)
                plt.plot(offset * cueExp, 
                        'r', 
                        label='bar', 
                        alpha=0.5,
                        linewidth=1
                        )

                condName = condId_to_State_map[condIdx]
                stimName = stimId_to_Song_map[stimIdx]

                title_str = '' #'='*50 + '\n'
                title_str += f'Subject {subjectID}: top EOG IC across trials\n'
                title_str += f'Task [{condIdx}]: {condName}\n'
                title_str += f'Stim [{stimIdx}]: {stimName}\n'
                title_str += f'Trials: {trials_to_plot}'
                plt.title(title_str)

                plt.xlabel('Time [Samples]')
                plt.ylabel('Trial #')
                plt.legend(loc='upper right', prop={'size': 6}) #, bbox_to_anchor=[1.5, 0.5])
                #print(title_str)

                pltIdx += 1

        plt.tight_layout()

In [ ]:
SAVE_FILTERED_BLINKS = False
if SAVE_FILTERED_BLINKS:
    filename = eog_peaks_filename.split('.npy')[0] + '_bpf.npy'
    np.save(filename, eog_peaks)

# Compare blink events before/after filtering

In [ ]:
# TODO

# Convolve with Rectangles

In [ ]:
def blink_atten( # update to use a convolution
        t, 
        peaks, 
        half_width
    ):
    peaks_sq = np.zeros_like(t)
    for peak in peaks:
        start_ndx = peak-half_width
        stop_ndx = peak+half_width
        if start_ndx < 0:
            start_ndx = 0
        if stop_ndx > len(t):
            stop_ndx = len(t)
        peaks_sq[start_ndx : stop_ndx] += 1
    return peaks_sq

Plot rectangular eye blink train:

In [ ]:
if 0:
    NOTES_Y_OFFSET = 12

    half_width = 10

    t = np.arange(start=0, stop=len(cueExp), step=1)

    peaks_sq_all = {}
    for subjectID in eog_peaks:
        peaks_sq_all[subjectID] = {}
        
        for stimIdx in [1, 2, 3, 4]: # top to bottom on the sheet music
            stimLabel = stimId_to_Song_map[stimIdx]
            peaks_sq_all[subjectID][stimLabel] = {}
            
            notes = t[stim[stimLabel]['notes']]
            beats = t[stim[stimLabel]['beats']>0]

            for condIdx in [1, 2]:
                condLabel = condId_to_State_map[condIdx]
                peaks_sq_all[subjectID][stimLabel][condLabel] = {}

                trials_to_plot = idxs[condLabel][stimLabel]
                    
                plt.figure(figsize=(8, 4))

                plt.plot(t[notes], 0*t[notes] + NOTES_Y_OFFSET, '.', label='notes')
                plt.plot(t[beats], 0*t[beats] + NOTES_Y_OFFSET, 'o', label='beats')

                trialCnt = 0
                for trialIdx in trials_to_plot:
                    peaks = eog_peaks[subjectID][trialIdx]['ica_eog_events']
                    peaks_sq = blink_atten(t, peaks, half_width)
                    
                    peaks_sq_all[subjectID][stimLabel][condLabel][trialIdx] = peaks_sq

                    #plt.plot(t, peaks_sq_rot + trialCnt)
                    line, = plt.plot(t, peaks_sq/2 + trialCnt)
                    plt.plot(t[peaks], 0*t[peaks] + trialCnt + 0.5, 'x', c=line.get_color())
                    plt.plot(t[peaks], 0*t[peaks] + trialCnt + 0.5, 'x', c=line.get_color())

                    trialCnt += 1

                plt.xlabel('Time [sample]')
                plt.ylabel('Trial #')
                plt.legend(loc='upper right')
                title_str = f'Subject{subjectID} | {condLabel} | {stimLabel}'
                plt.title(title_str)

Examine eog_peaks structure as loaded from .npy object:

In [ ]:
sub = 1 # [1, 3, 7, 11]
condLabel = 'Imagery'
stimLabel = 'chor-019'

trials = idxs[condLabel][stimLabel] # [2, 8, 14, 15, 26, 27, 29, 36, 47, 48, 52, 3, 6, 11, 28, 33, 34, 37, 51, 58, 62, 70, 7, 10, 21, 23, 30, 35, 38, 39, 66, 74, 86, 12, 17, 24, 44, 45, 49, 54, 55, 56, 64, 76, 18, 19, 31, 32, 41, 57, 61, 69, 79, 84, 87, 1, 25, 42, 43, 46, 59, 60, 67, 68, 73, 85, 0, 13, 16, 22, 40, 63, 71, 75, 77, 78, 83, 4, 5, 9, 20, 50, 53, 65, 72, 80, 81, 82])

for trial in trials:
    blinks = eog_peaks[sub][trial]['ica_eog_events']
    print(trial, blinks)

## Read in images of the score

In [ ]:
if 0:
    import matplotlib.image as mpimg
    import matplotlib as mpl 

    # from stackexchange
    imgs = {}

    start_px = 50
    for bar in range(8):
        img = mpimg.imread('data/Score.png')
        
        stop_px = start_px + 70
        imgs[bar] = img[start_px:stop_px, 10:-15].copy()
        
        # offset to the next bar
        start_px += 100
        if bar%2 == 1:
            start_px += 100
            
    %matplotlib widget 

    for bar in range(8):
        plt.figure()
        plt.imshow(imgs[bar])
        plt.show()

# Info Theory Metric

In [ ]:
condId_to_State_map

In [ ]:
stimLabel_to_id_map.keys()

In [ ]:
def coding_cost(
        p, q,
        SMALL_NUMBER = 1e-20, #np.nextafter(0, 1)
        SQ_SANITY_CHECK = False,
        NORM = 'sum',
    ):
    p += SMALL_NUMBER
    q += SMALL_NUMBER

    if NORM == 'sum':
        #print(f'p: {p.shape}, q: {q.shape}')
        p /= np.sum(p) # normalize the area
        q /= np.sum(q) # normalize the area
    elif NORM == 'max':
        p /= np.max(p) # normalize the area
        q /= np.max(q) # normalize the area
    elif NORM == 'none':
        p /= 1
        q /= 1

        if 0:
            p[(1-p)<SMALL_NUMBER] -= SMALL_NUMBER # if any p_i == 1, make it slightly smaller to prevent division by log(1) = 0
            q[(1-q)<SMALL_NUMBER] -= SMALL_NUMBER # if any q_i == 1, make it slightly smaller to prevent division by log(1) = 0
            
            p[p<SMALL_NUMBER] += SMALL_NUMBER # if any p_i == 0, make it slightly bigger to prevent division by log(1) = 0
            q[q<SMALL_NUMBER] += SMALL_NUMBER # if any q_i == 0, make it slightly bigger to prevent division by log(1) = 0
    
    # Rescale to 0-1
    if 0:
        norm_factor = np.max(
            (np.abs(p), np.abs(q))
        )
        p /= norm_factor
        q /= norm_factor

    info = 0
    for i in range(len(p)):
        p_i = p[i]
        q_i = q[i] 
        if SQ_SANITY_CHECK:
            info += (p_i - q_i)**2
        else:
            info += -q_i * np.log2(p_i, where=p_i>0) 
            info += -(1-q_i)*np.log2(1-p_i, where=(1-p_i)>0)
    
    info /= len(p)
    return info, p, q

$$
qlog(p) + (1-q)log(p)
$$

$$
plog(q) + (1-p)log(q)
$$

# Duration of a quarter note in #samples (for timescales)

In [ ]:
bpm = 100
bps = bpm/60
n_quarter_note = int(bps*fs)
print(n_quarter_note)

# Split training ($p$) and test ($q$) trials

To Do:
- Split p/q 5:5, keep one trial aside for decoding testing
- Compare coding cost against a uniform p (flat blink prob across of 1/1803)

In [ ]:
PLOT_DISTROS = False # True # visualize p and q distros
SQ_SANITY_CHECK = False 

NUM_SUBJECTS = len(eog_peaks)
TEST_STIM_LABELS = ['chor-038', 'chor-096', 'chor-101', 'chor-019']
condLabels = ['Listening', 'Imagery'] #['Listening'] # consider only listening

NUM_TRAINING_TRIALS = 6 #10

TIMESCALES = [
    n_quarter_note/4,   # 1/16
    n_quarter_note/2,   # 1/8
    n_quarter_note,     # 1/4
    n_quarter_note*2,   # 1/2
    n_quarter_note*4,   # 1
    n_quarter_note*8,   # 2
    n_quarter_note*16,   # 4
]
half_width_sweep = [] #collects corresponding half width durations

t = np.arange(start=0, stop=len(cueExp), step=1)

AVG_LOOPS = 1
train_trial_avg = {}
test_trial_avg = {} # data for test set?

for n_loop in range(AVG_LOOPS):
    for n_note in TIMESCALES:
        
        half_width = int(n_note/2)
        half_width_sweep.append(half_width)
        
        if not half_width in train_trial_avg:
            train_trial_avg[half_width] = {}
            test_trial_avg[half_width] = {}

        if PLOT_DISTROS:
            plt.figure(figsize = (20, 6*NUM_SUBJECTS))
            pltIdx = 0

        for subjectID in eog_peaks.keys(): #allTrialsForNullDistro:
            if not subjectID in train_trial_avg[half_width]:
                train_trial_avg[half_width][subjectID] = {}
                test_trial_avg[half_width][subjectID] = {}

            for condLabel in condLabels:
                if not condLabel in train_trial_avg[half_width][subjectID]:
                    train_trial_avg[half_width][subjectID][condLabel] = {}
                    test_trial_avg[half_width][subjectID][condLabel] = {}
                
                for stimLabel in TEST_STIM_LABELS:
                    if not stimLabel in train_trial_avg[half_width][subjectID][condLabel]:
                        train_trial_avg[half_width][subjectID][condLabel][stimLabel] = {}
                        test_trial_avg[half_width][subjectID][condLabel][stimLabel] = {}

                    if not n_loop in train_trial_avg[half_width][subjectID][condLabel][stimLabel]:
                        train_trial_avg[half_width][subjectID][condLabel][stimLabel][n_loop] = np.zeros_like(t, dtype=np.float64)
                        test_trial_avg[half_width][subjectID][condLabel][stimLabel][n_loop] = np.zeros_like(t, dtype=np.float64)

                    # Split trials into train-test sets
                    allMatchingTrials = [x for x in idxs[condLabel][stimLabel]] #peaks_sq_all[subjectID][stimLabel][condLabel].keys()]
                    allMatchingTrials = np.array(allMatchingTrials)
                    np.random.shuffle(allMatchingTrials)
                    
                    #print('All trials:', allMatchingTrials)
                    trainTrialIdxs = allMatchingTrials[:NUM_TRAINING_TRIALS]
                    print('Train trials:', trainTrialIdxs)
                    testTrialIdxs = allMatchingTrials[NUM_TRAINING_TRIALS:]
                    print('Test trials:', testTrialIdxs)

                    # Get the distro for training trials
                    for trainTrial in trainTrialIdxs:
                        train_peaks = eog_peaks[subjectID][trainTrial]['ica_eog_events']
                        train_peaks_sq = blink_atten(t, train_peaks, half_width)
                        train_trial_avg[half_width][subjectID][condLabel][stimLabel][n_loop] += train_peaks_sq / NUM_TRAINING_TRIALS


                    # Now get the distro for test trials
                    for testTrial in testTrialIdxs:
                        test_peaks = eog_peaks[subjectID][testTrial]['ica_eog_events']
                        test_peaks_sq = blink_atten(t, test_peaks, half_width)
                        #val_peaks_sq = peaks_sq_all[subjectID][stimLabel][condLabel][testTrial]
                        test_trial_avg[half_width][subjectID][condLabel][stimLabel][n_loop] += test_peaks_sq / len(testTrialIdxs)

                    if PLOT_DISTROS:
                        notes = t[stim[stimLabel]['notes']]
                        beats = t[stim[stimLabel]['beats']>0]
                        
                        p = np.copy(train_trial_avg[half_width][subjectID][condLabel][stimLabel][n_loop])
                        q = np.copy(test_trial_avg[half_width][subjectID][condLabel][stimLabel][n_loop])
                        info = coding_cost(p, q, SQ_SANITY_CHECK=SQ_SANITY_CHECK)

                        pltIdx += 1
                        ax = plt.subplot(len(eog_peaks.keys()) * len(condLabels), 4, pltIdx)

                        train_line, = ax.plot(
                            t/fs, 
                            #peaks_sq_baseline, 
                            p,
                            #'.', 
                            label=f'Training avg.', 
                            c='k',
                            linewidth=1,
                            #alpha=0.1,
                        )

                        test_line, = ax.plot(
                            t/fs, 
                            #test_trial_avg[subjectID][stimLabel], 
                            q, 
                            #'.', 
                            label=f'Test avg.', 
                            c='r',
                            linewidth=1,
                            #alpha=0.1,
                        )

                        title_str = 'Blink Dur: {:.3f} s'.format(2 * half_width * 1/fs)
                        title_str+= f', sub-{subjectID}, {condLabel}, {stimLabel}'
                        title_str+= f'\nTraining Trials\t(for $p_i$): {trainTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                        title_str+= f'\nTest Trials\t(for $q_i$): {testTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                        if SQ_SANITY_CHECK:
                            title_str+= '\n$ 1/N \sum (p_i - q_i)^2$ = {:.5f}'.format(info)
                        else:
                            title_str+= '\n$-1/N \sum q_i log p_i + (1-q_i) log (1-p_i)$ = {:.5f}'.format(info)
                        ax.set_title(title_str)

                        ax.set_xlabel('Time [s]')
                        #ax.set_ylim([0, .01])

    if PLOT_DISTROS:
        plt.tight_layout()
        plt.show()

In [ ]:
def plot_info(results_matrix, ax, rowLabels, colLabels, title= "", vmin = None, vmax=None, fontsize=4, cmap='magma'):
    # https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    # fig, ax = plt.subplots(figsize = figsize)
    im = ax.imshow(results_matrix, cmap=cmap, interpolation=None, vmin=vmin, vmax=vmax)
    
    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(colLabels)), labels=colLabels)
    ax.set_yticks(np.arange(len(rowLabels)), labels=rowLabels)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_yticklabels(), fontsize=fontsize)
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
            rotation_mode="anchor", fontsize=fontsize)
    '''
    # Loop over data dimensions and create text annotations.
    for i in range(N):
        for j in range(N):
            text = ax.text(j, i, results_matrix[i, j],
                        ha="center", va="center", color="w")
    '''
    ax.set_title(title)
    plt.colorbar(im, ax=ax)
    plt.tight_layout()
    #plt.show()

def plot_pq(ax, p, q, t, fs, title_str = ''):
    #ax = plt.subplot(len(eog_peaks.keys()) * len(condLabels), 4, pltIdx)

    train_line, = ax.plot(
        t/fs, 
        #peaks_sq_baseline, 
        p,
        #'.', 
        label=f'p', 
        c='k',
        linewidth=1,
        #alpha=0.1,
    )

    test_line, = ax.plot(
        t/fs, 
        #test_trial_avg[subjectID][stimLabel], 
        q, 
        #'.', 
        label=f'q', 
        c='r',
        linewidth=1,
        #alpha=0.1,
    )
    #ax.legend()
    ax.set_title(title_str)
    ax.set_xlabel('Time [s]')

In [ ]:
stimId_to_Song_map

In [ ]:
train_trial_avg.keys()

In [ ]:
half_width_sweep

In [ ]:
PLOT_PQ = not True

subjectIDs = [1, 6, 7, 11, 17]
half_widths = [13, 26] #, 53, 106]
condLabels = ['Listening', 'Imagery'] #list(train_trial_avg[half_widths[0]][subjectIDs[0]].keys())
stimLabels = list(train_trial_avg[half_widths[0]][subjectIDs[0]][condLabels[0]].keys())
NORMS = ['sum'] #, 'none'] #['sum'] #, 'max']
sq_checks = [False] #[True, False]
P_BASELINE_CHECK = [True, False]
P_TRAIN = True

for P_BASELINE in P_BASELINE_CHECK:
    for SQ_SANITY_CHECK in sq_checks:
        for Q_TRAIN in [False]: #[True, False]:
            for half_width in half_widths:
                for NORM in NORMS:
                    
                    rowLabels = []
                    colLabels = []

                    N = len(subjectIDs)*len(condLabels)*len(stimLabels)
                    results_matrix = np.zeros(
                        shape=(N, N),
                    )
                    p_norms = {}
                    q_norms = {}

                    for i in range(N):
                        p_norms[i] = {}
                        q_norms[i] = {}

                        for j in range(N):

                            subjectID1 = subjectIDs[int(i / (len(condLabels)*len(stimLabels)))]
                            subjectID2 = subjectIDs[int(j / (len(condLabels)*len(stimLabels)))]

                            condLabel1 = condId_to_State_map[1 + int(i / len(stimLabels))%len(condLabels)]
                            condLabel2 = condId_to_State_map[1 + int(j / len(stimLabels))%len(condLabels)]

                            stimLabel1 = stimId_to_Song_map[1 + i % len(stimLabels)] #% (len(subjectIDs)*len(condLabels))]
                            stimLabel2 = stimId_to_Song_map[1 + j % len(stimLabels)] #% (len(subjectIDs)*len(condLabels))]

                            if i == 0:
                                colLabels.append(f'{subjectID2} / {condLabel2} / {stimLabel2}')


                            p_norms[i][j] = np.zeros_like(t, dtype=np.float64)
                            q_norms[i][j] = np.zeros_like(t, dtype=np.float64)
                            
                            for n_loop in range(AVG_LOOPS):
                                if P_BASELINE:
                                    p = np.ones_like(t)/len(t)
                                else: 
                                    if P_TRAIN:
                                        p = train_trial_avg[half_width][subjectID1][condLabel1][stimLabel1][n_loop]
                                    else:
                                        p = test_trial_avg[half_width][subjectID1][condLabel1][stimLabel1][n_loop]

                                if Q_TRAIN:
                                    q = train_trial_avg[half_width][subjectID2][condLabel2][stimLabel2][n_loop]
                                else:
                                    q = test_trial_avg[half_width][subjectID2][condLabel2][stimLabel2][n_loop]

                                cost, p_norm, q_norm = coding_cost(
                                    np.copy(p), 
                                    np.copy(q), 
                                    SMALL_NUMBER=1e-20,
                                    SQ_SANITY_CHECK=SQ_SANITY_CHECK,
                                    NORM=NORM,
                                ) 
                                #print(sum(p_norm), sum(q_norm)) # verified approx 1
                                results_matrix[i, j] += cost / AVG_LOOPS
                                p_norms[i][j] += p_norm / AVG_LOOPS
                                q_norms[i][j] += q_norm / AVG_LOOPS

                                #print(f'{subjectID1}/{condLabel1}/{stimLabel1} vs. {subjectID2}/{condLabel2}/{stimLabel2}\t{results_matrix[i, j]}\t{p}\t{q}')

                        # update after IDs have been calculated above:
                        rowLabels.append(f'{subjectID1} / {condLabel1} / {stimLabel1}')

                    title = 'Blink Time: {:.3f} ms = {:.2f} qtr notes @ 100 bpm'.format(
                        (2 * half_width / fs)*1000,
                        (2 * half_width / fs) / (1/bps),
                    )
                    title += f'\nNorm by {NORM}\n'
                    
                    if SQ_SANITY_CHECK:
                        title += 'coding cost: sq'
                    else:
                        title += 'coding cost: log'
                    title += '\n'

                    if P_BASELINE:
                        title += 'p: baseline, $\sum$p = {:.2f}\n'.format(sum(p_norm))
                    else:
                        if P_TRAIN:
                            title += f'p [{NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: train'
                        else:
                            title += f'p [{11-NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: test'
                        title += '\n'
                    
                    if Q_TRAIN:
                        title += f'q [{NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: train'
                    else:
                        title += f'q [{11-NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: test'
                    
                    figsize = (N*0.4, N*0.4/2)

                    fig, ax = plt.subplots(figsize=figsize)
                    plot_info(results_matrix,
                            ax, 
                            rowLabels, colLabels, title, 
                            vmin=0,
                            vmax=0.05, #10,
                            fontsize = 10,
                            )
                    
                    if PLOT_PQ:
                        fig, axs = plt.subplots(N, N, figsize=figsize, sharex=True, sharey=True)

                        for i in range(N):
                            for j in range(N):
                                title_str = 'Blink Dur: {:.3f} s'.format(2 * half_width * 1/fs)
                                title_str+= f', sub-{subjectID}, {condLabel}, {stimLabel}'
                                title_str+= f'\nTraining Trials\t(for $p_i$): {trainTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                                title_str+= f'\nTest Trials\t(for $q_i$): {testTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                                if SQ_SANITY_CHECK:
                                    title_str+= '\n$ 1/N \sum (p_i - q_i)^2$ = {:.5f}'.format(results_matrix[i, j])
                                else:
                                    title_str+= '\n$-1/N \sum q_i log p_i + (1-q_i) log (1-p_i)$ = {:.5f}'.format(results_matrix[i, j])

                                plot_pq(axs[i][j], p_norms[i][j], q_norms[i][j], t, fs)
                        plt.tight_layout()
                    plt.show()

In [ ]:
PLOT_PQ = False

subjectIDs = [1, 6, 7, 11, 17] #[1, 3, 7, 11]
half_widths = [13, 26] #, 53, 106]
condLabels = ['Listening', 'Imagery'] #list(train_trial_avg[half_widths[0]][subjectIDs[0]].keys())
stimLabels = list(train_trial_avg[half_widths[0]][subjectIDs[0]][condLabels[0]].keys())
NORMS = ['sum'] #, 'none'] #['sum'] #, 'max']
sq_checks = [False] #[True, False]
P_BASELINE_CHECK = [True, False]
P_TRAIN = True

for P_BASELINE in P_BASELINE_CHECK:
    for SQ_SANITY_CHECK in sq_checks:
        for Q_TRAIN in [False]: #[True, False]:
            for half_width in half_widths:
                for NORM in NORMS:
                    rowLabels = []
                    colLabels = []

                    N = len(subjectIDs)*len(condLabels)*len(stimLabels)
                    results_matrix = np.zeros(
                        shape=(N, N),
                    )
                    p_norms = {}
                    q_norms = {}

                    for i in range(N):
                        p_norms[i] = {}
                        q_norms[i] = {}

                        for j in range(N):
                            subjectID1 = subjectIDs[i % len(subjectIDs)] #[int(i / (len(condLabels)*len(stimLabels)))]
                            subjectID2 = subjectIDs[j % len(subjectIDs)] #[int(j / (len(condLabels)*len(stimLabels)))]
                            condLabel1 = condId_to_State_map[1 + int(i / len(stimLabels))%len(condLabels)]
                            condLabel2 = condId_to_State_map[1 + int(j / len(stimLabels))%len(condLabels)]
                            stimLabel1 = stimId_to_Song_map[1 + int(i / (len(condLabels)*len(subjectIDs)))] #[1 + (i % 4)]
                            stimLabel2 = stimId_to_Song_map[1 + int(j / (len(condLabels)*len(subjectIDs)))] #[1 + (j % 4)]

                            if i == 0:
                                colLabels.append(f'{subjectID2} / {condLabel2} / {stimLabel2}')


                            p_norms[i][j] = np.zeros_like(t, dtype=np.float64)
                            q_norms[i][j] = np.zeros_like(t, dtype=np.float64)
                            
                            for n_loop in range(AVG_LOOPS):
                                if P_BASELINE:
                                    p = np.ones_like(t)/len(t)
                                else: 
                                    if P_TRAIN:
                                        p = train_trial_avg[half_width][subjectID1][condLabel1][stimLabel1][n_loop]
                                    else:
                                        p = test_trial_avg[half_width][subjectID1][condLabel1][stimLabel1][n_loop]

                                if Q_TRAIN:
                                    q = train_trial_avg[half_width][subjectID2][condLabel2][stimLabel2][n_loop]
                                else:
                                    q = test_trial_avg[half_width][subjectID2][condLabel2][stimLabel2][n_loop]

                                cost, p_norm, q_norm = coding_cost(
                                    np.copy(p), 
                                    np.copy(q), 
                                    SMALL_NUMBER=1e-20,
                                    SQ_SANITY_CHECK=SQ_SANITY_CHECK,
                                    NORM=NORM,
                                ) 
                                results_matrix[i, j] += cost / AVG_LOOPS
                                p_norms[i][j] += p_norm / AVG_LOOPS
                                q_norms[i][j] += q_norm / AVG_LOOPS

                                #print(f'{subjectID1}/{condLabel1}/{stimLabel1} vs. {subjectID2}/{condLabel2}/{stimLabel2}\t{results_matrix[i, j]}\t{p}\t{q}')

                        # update after IDs have been calculated above:
                        rowLabels.append(f'{subjectID1} / {condLabel1} / {stimLabel1}')

                    title = 'Blink Time: {:.3f} ms = {:.2f} qtr note beats @ 100 bpm'.format(
                        (2 * half_width / fs)*1000,
                        (1/bps) / (2 * half_width / fs),
                    )
                    title += f'\nNorm by {NORM}\n'
                    
                    if SQ_SANITY_CHECK:
                        title += 'coding cost: sq'
                    else:
                        title += 'coding cost: log'
                    title += '\n'

                    if P_BASELINE:
                        title += 'p: baseline, $\sum$p = {:.2f}\n'.format(sum(p_norm))
                    else:
                        if P_TRAIN:
                            title += f'p [{NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: train'
                        else:
                            title += f'p [{11-NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: test'
                        title += '\n'
                    
                    if Q_TRAIN:
                        title += f'q [{NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: train'
                    else:
                        title += f'q [{11-NUM_TRAINING_TRIALS}/11, #avg: {AVG_LOOPS}]: test'
                    
                    figsize = (N*0.4, N*0.4/2)

                    fig, ax = plt.subplots(figsize=figsize)
                    plot_info(results_matrix,
                            ax, 
                            rowLabels, colLabels, title, 
                            vmin = 0,
                            vmax=0.05, #10,
                            fontsize = 10,
                            )
                    
                    if PLOT_PQ:
                        fig, axs = plt.subplots(N, N, figsize=figsize, sharex=True, sharey=True)

                        for i in range(N):
                            for j in range(N):
                                title_str = 'Blink Dur: {:.3f} s'.format(2 * half_width * 1/fs)
                                title_str+= f', sub-{subjectID}, {condLabel}, {stimLabel}'
                                title_str+= f'\nTraining Trials\t(for $p_i$): {trainTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                                title_str+= f'\nTest Trials\t(for $q_i$): {testTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                                if SQ_SANITY_CHECK:
                                    title_str+= '\n$ 1/N \sum (p_i - q_i)^2$ = {:.5f}'.format(results_matrix[i, j])
                                else:
                                    title_str+= '\n$-1/N \sum q_i log p_i + (1-q_i) log (1-p_i)$ = {:.5f}'.format(results_matrix[i, j])

                                plot_pq(axs[i][j], p_norms[i][j], q_norms[i][j], t, fs)
                        plt.tight_layout()
                    plt.show()

In [ ]:
PLOT_PQ = False

subjectIDs = [1, 3, 7, 11]
half_widths = [13, 26, 53, 106]
condLabels = ['Listening', 'Imagery'] #list(train_trial_avg[half_widths[0]][subjectIDs[0]].keys())
stimLabels = list(train_trial_avg[half_widths[0]][subjectIDs[0]][condLabels[0]].keys())
NORMS = ['sum'] #, 'none'] #['sum'] #, 'max']
sq_checks = [False] #[True, False]

P_TRAIN = True
for SQ_SANITY_CHECK in sq_checks:
    for Q_TRAIN in [False]: # [True, False]:
        for half_width in half_widths:
            for NORM in NORMS:
                rowLabels = []
                colLabels = []

                N = len(subjectIDs)*len(condLabels)*len(stimLabels)
                results_matrix = np.zeros(
                    shape=(N, N),
                )
                p_norms = {}
                q_norms = {}

                i = 0
                for i in range(N):
                    p_norms[i] = {}
                    q_norms[i] = {}

                    for j in range(N):
                        subjectID1 = subjectIDs[i % len(subjectIDs)] #[int(i / (len(condLabels)*len(stimLabels)))]
                        subjectID2 = subjectIDs[j % len(subjectIDs)] #[int(j / (len(condLabels)*len(stimLabels)))]
                        condLabel1 = condId_to_State_map[1 + int(i / len(stimLabels))%len(condLabels)]
                        condLabel2 = condId_to_State_map[1 + int(j / len(stimLabels))%len(condLabels)]
                        stimLabel1 = stimId_to_Song_map[1 + int(i / (len(condLabels)*len(stimLabels)))] #[1 + (i % 4)]
                        stimLabel2 = stimId_to_Song_map[1 + int(j / (len(condLabels)*len(stimLabels)))] #[1 + (j % 4)]

                        if i == 0:
                            colLabels.append(f'{subjectID2} / {condLabel2} / {stimLabel2}')

                        if P_TRAIN:
                            p = train_trial_avg[half_width][subjectID1][condLabel1][stimLabel1]
                        else:
                            p = test_trial_avg[half_width][subjectID1][condLabel1][stimLabel1]

                        if Q_TRAIN:
                            q = train_trial_avg[half_width][subjectID2][condLabel2][stimLabel2]
                        else:
                            q = test_trial_avg[half_width][subjectID2][condLabel2][stimLabel2]

                        results_matrix[i, j], p_norms[i][j], q_norms[i][j] = coding_cost(
                            np.copy(p), 
                            np.copy(q), 
                            SMALL_NUMBER=1e-20,
                            SQ_SANITY_CHECK=SQ_SANITY_CHECK,
                            NORM=NORM,
                        )

                        #print(f'{subjectID1}/{condLabel1}/{stimLabel1} vs. {subjectID2}/{condLabel2}/{stimLabel2}\t{results_matrix[i, j]}\t{p}\t{q}')

                    # update after IDs have been calculated above:
                    rowLabels.append(f'{subjectID1} / {condLabel1} / {stimLabel1}')

                title = 'Blink Time: {:.3f} ms = {:.2f} qtr note beats @ 100 bpm'.format(
                    2 * 1000 * half_width * 1/fs,
                    (2 * half_width / (bps*fs)),
                )
                title += f'\nNorm by {NORM}\n'
                
                if SQ_SANITY_CHECK:
                    title += 'coding cost: sq'
                else:
                    title += 'coding cost: log'
                title += '\n'

                if P_TRAIN:
                    title += f'p [{NUM_TRAINING_TRIALS}/11]: train'
                else:
                    title += f'p [{11-NUM_TRAINING_TRIALS}/11]: test'
                title += '\n'
                
                if Q_TRAIN:
                    title += f'q [{NUM_TRAINING_TRIALS}/11]: train'
                else:
                    title += f'q [{11-NUM_TRAINING_TRIALS}/11]: test'
                
                figsize = (N*.6, .6*N/2)

                fig, ax = plt.subplots(figsize=figsize)
                plot_info(results_matrix,
                          ax, 
                          rowLabels, colLabels, title, 
                          #vmax=0.05, #10,
                          fontsize = 10,
                        )
                
                if PLOT_PQ:
                    fig, axs = plt.subplots(N, N, figsize=figsize, sharex=True, sharey=True)

                    for i in range(N):
                        for j in range(N):
                            title_str = 'Blink Dur: {:.3f} s'.format(2 * half_width * 1/fs)
                            title_str+= f', sub-{subjectID}, {condLabel}, {stimLabel}'
                            title_str+= f'\nTraining Trials\t(for $p_i$): {trainTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                            title_str+= f'\nTest Trials\t(for $q_i$): {testTrialIdxs}' # {NUM_TRAINING_TRIALS}'
                            if SQ_SANITY_CHECK:
                                title_str+= '\n$ 1/N \sum (p_i - q_i)^2$ = {:.5f}'.format(results_matrix[i, j])
                            else:
                                title_str+= '\n$-1/N \sum q_i log p_i + (1-q_i) log (1-p_i)$ = {:.5f}'.format(results_matrix[i, j])

                            plot_pq(axs[i][j], p_norms[i][j], q_norms[i][j], t, fs)
                    plt.tight_layout()

                plt.show()

# Mega Matrix


In [ ]:
half_widths = list(train_trial_avg.keys())
subjectIDs = list(train_trial_avg[half_widths[0]].keys())
condLabels = list(train_trial_avg[half_widths[0]][subjectIDs[0]].keys())
stimLabels = list(train_trial_avg[half_widths[0]][subjectIDs[0]][condLabels[0]].keys())

for half_width in half_widths:

    rowLabels = []
    colLabels = []

    N = len(subjectIDs)*len(condLabels)*len(stimLabels)
    results_matrix = np.zeros(
        shape=(N, N),
    )

    i = 0
    for i in range(N):

        for j in range(N):
            subjectID1 = 1 + int(i / (len(condLabels)*len(stimLabels)))
            subjectID2 = 1 + int(j / (len(condLabels)*len(stimLabels)))
            condLabel1 = condId_to_State_map[1 + int(i / len(stimLabels))%2]
            condLabel2 = condId_to_State_map[1 + int(j / len(stimLabels))%2]
            stimLabel1 = stimId_to_Song_map[1 + (i % 4)]
            stimLabel2 = stimId_to_Song_map[1 + (j % 4)]

            if i == 0:
                colLabels.append(f'{subjectID2} / {condLabel2} / {stimLabel2}')

            p = train_trial_avg[half_width][subjectID1][condLabel1][stimLabel1]
            #q = train_trial_avg[half_width][subjectID2][condLabel2][stimLabel2]
            q = test_trial_avg[half_width][subjectID2][condLabel2][stimLabel2]

            results_matrix[i, j] = coding_cost(
                np.copy(p), 
                np.copy(q), 
                SMALL_NUMBER=1e-20,
                SQ_SANITY_CHECK=False, #True,
            )

            #print(f'{subjectID1}/{condLabel1}/{stimLabel1} vs. {subjectID2}/{condLabel2}/{stimLabel2}\t{results_matrix[i, j]}\t{p}\t{q}')

        rowLabels.append(f'{subjectID1} / {condLabel1} / {stimLabel1}')

    title = 'Blink Time: {:.3f} ms = {:.2f} qtr note beats @ 100 bpm'.format(
        2 * 1000 * half_width * 1/fs,
        (2 * half_width / (bps*fs))
    )
    plot_info(results_matrix, rowLabels, colLabels, title, 
              #vmax=10, 
              figsize = (16, 8),
              fontsize = 10,
            )

Visualizing log values:

In [ ]:
plt.figure(100)
plt.clf()

plt.subplot(3, 1, 1)
plt.plot(p, label='$p$')
plt.plot(1-p, label='$1-p$')
plt.legend()
plt.ylim([-0.1, 1.1])

plt.subplot(3, 1, 2)
plt.plot(np.log2(p + np.nextafter(0, 1)),
         label = '$log_2(p)$')
plt.plot(np.log2(1-p),
         label = '$log_2(1-p)$')
plt.legend()
plt.ylim([-10, 1])

plt.subplot(3, 1, 3)
plt.plot(-p * np.log2(p + np.nextafter(0, 1)),
         label = '$-p.log_2(p)$')
plt.plot(-(1-p) * np.log2(1-p),
         label = '$-(1-p)log_2(1-p)$')
plt.legend()

plt.ylim([-.1, 1.1])
plt.show()

In [ ]:
0.5*np.log2(0.5)*len(p)

In [ ]:
np.max((p_i, q_i))

# (old) Permutations for Null Distros

In [ ]:
NUM_LOOPS = 10000
NUM_TRIALS = 11
TEST_STIM_LABELS = ['chor-038', 'chor-096', 'chor-101', 'chor-019']
NOTES_Y_OFFSET = 1

PLOT_POINT_CLOUD = False # this takes a lot of memory => slow down

LOAD_DISTRO = not True
SAVE_DISTRO = not LOAD_DISTRO 

fs = 64

baseline = {} # this will be the null distro
test_trial_avg = {}
peaks_sq_baseline_distro = {}

t = np.arange(start=0, stop=len(cueExp), step=1)

plt.figure(figsize = (32, 16))
pltIdx = 0

rowIdx = 1
for subjectID in allTrialsForNullDistro:
    if LOAD_DISTRO:
        peaks_sq_baseline_distro_samples = np.load(
            f"peaks_sq_baseline_distro_samples_sub{subjectID}_{','.join(condLabels)}_{NUM_LOOPS}.npy"
        )
        print(peaks_sq_baseline_distro_samples.shape)

    else:
        peaks_sq_baseline_distro[subjectID] = []

        # Draw NUM_TRIALS from all trials, and loop NUM_LOOPS times
        for n in range(NUM_LOOPS):
            peaks_sq_baseline = np.zeros_like(t, dtype=np.float64)
            
            randTrialIdxs = np.random.random_integers(
                low=0, 
                high=len(allTrialsForNullDistro[subjectID]) -1, 
                size=NUM_TRIALS,
            )

            for randn in randTrialIdxs: # decide which of 44 trials to draw
                #print(randn)
                peaks_sq_baseline += allTrialsForNullDistro[subjectID][randn]

            peaks_sq_baseline /= NUM_TRIALS # this gives us the average of 11 randomly selected trials
            peaks_sq_baseline_distro[subjectID].append(peaks_sq_baseline)

        #peaks_sq_baseline_distro_samples = np.empty_like(t) # this adds an extra row?
        i = 0
        for samples in peaks_sq_baseline_distro[subjectID]:
            if i == 0:
                peaks_sq_baseline_distro_samples = samples
            else:
                peaks_sq_baseline_distro_samples = np.vstack(
                    (peaks_sq_baseline_distro_samples, samples)
                )
            i+=1
        print(peaks_sq_baseline_distro_samples.shape)

        # sort the samples for every time stamp to get the 95% interval
        peaks_sq_baseline_distro_samples.sort(axis=0)

        # Save to memory!
        if SAVE_DISTRO:
            np.save(
                file=f"peaks_sq_baseline_distro_samples_sub{subjectID}_{','.join(condLabels)}_{NUM_LOOPS}",
                arr=peaks_sq_baseline_distro_samples,
            )

    test_trial_avg[subjectID] = {}
    for stimLabel in TEST_STIM_LABELS:
        pltIdx += 1
        ax = plt.subplot(len(allTrialsForNullDistro), 4+1, pltIdx)

        # Plot null distro point cloud
        if PLOT_POINT_CLOUD:
            #for samples in peaks_sq_baseline_distro[subjectID]:
            for row in range(peaks_sq_baseline_distro_samples.shape[0]):
                line, = ax.plot(
                    t/fs, 
                    peaks_sq_baseline_distro_samples[row], 
                    '.', 
                    #label='validation blinks', 
                    c='k',
                    alpha=0.1,
                    markersize=.1,
                    label = 'Max[null]'
                )
        else:
            max_line, = ax.plot(
                t/fs, 
                peaks_sq_baseline_distro_samples[-1, :], 
                '.', 
                #label='validation blinks', 
                c='k',
                alpha=0.1,
                #markersize=.1,
            )


        # Plot null distro CI intervals
        null_distro_95 = peaks_sq_baseline_distro_samples[int(.95*NUM_LOOPS), :]

        null_line, = ax.plot(
            t/fs, 
            null_distro_95, 
            #marker = '_',
            #ls = '', 
            #label='validation blinks', 
            c='c',
            #alpha=0.1,
        )

        ax.fill_between(
            t/fs, 
            peaks_sq_baseline_distro_samples[int(.95*NUM_LOOPS), :], 
            0, 
            color=null_line.get_color(), 
            alpha=.3,
            label = 'Null Distribution'
        )

        test_trial_avg[subjectID][stimLabel] = np.zeros_like(t, dtype=np.float64)

        notes = t[stim[stimLabel]['notes']]
        beats = t[stim[stimLabel]['beats']>0]
        
        for condLabel in condLabels:

            for trialIdx in peaks_sq_all[subjectID][stimLabel][condLabel]:
                val_peaks_sq = peaks_sq_all[subjectID][stimLabel][condLabel][trialIdx]
                test_trial_avg[subjectID][stimLabel] += val_peaks_sq
                
            test_trial_avg[subjectID][stimLabel] /= len(peaks_sq_all[subjectID][stimLabel][condLabel])

            test_distro = test_trial_avg[subjectID][stimLabel]

            avg_line, = ax.plot(
                t/fs, 
                test_distro, 
                #'.', 
                #label=f'Trial avg. for {stimLabel}', 
                c='r',
                #alpha=0.1,
            )

            ax.fill_between(
                t/fs, 
                test_distro, 
                0, 
                color=avg_line.get_color(), 
                alpha=.3,
                label=f'Trial avg. for {stimLabel}', 
            )

            significant_samples = (test_distro - null_distro_95) > 0

            sig_line, = ax.plot(
                t/fs,
                significant_samples,
                color = 'yellow',
                alpha = 0.3,
            )
            ax.fill_between(
                t/fs, 
                significant_samples, 
                0, 
                color=sig_line.get_color(), 
                label = 'Test > 95%ile of null',
                alpha=.3,
            )

            score = 100 * np.sum(significant_samples) / len(significant_samples)

        ax.plot(t[notes]/fs, 0*t[notes] + NOTES_Y_OFFSET, '.', label='notes')
        beat_dots, = ax.plot(t[beats]/fs, 0*t[beats] + NOTES_Y_OFFSET, 'o', label='beats')
        for beat in beats:
            ax.axvline(
                x=t[beat]/fs, 
                ls='-', # solid line
                alpha=0.2, 
                c=beat_dots.get_color()
            )

        #ax_base = ax.twinx()
        #ax_base.plot(t/fs, peaks_sq_baseline, label='baseline') #, c=line.get_color())
        
        title_str = f'Subject{subjectID} | {condLabel} | {stimLabel} vs. Null'
        title_str+= f'\n#Trials Per Avg: {NUM_TRIALS} | #Loops: {NUM_LOOPS}'
        title_str+= '\nTest > Null for {:.2f}% time'.format(score)
        ax.set_title(title_str)

        ax.set_xlabel('Time [s]')
        ax.set_ylim([-.1, 1.1])

        if (pltIdx % 5) == 1: # first column
            ax.legend(loc='upper right', bbox_to_anchor=[-0.1, 1])

        # Plot the significance curves for all songs in the last column
        axLastCol = plt.subplot(
            len(allTrialsForNullDistro), # nrows = no. of subjects
            4+1, # cols = no. of songs + one "bar codes" plot
            5*rowIdx, # select the last column of this row
        )
        sig_line, = axLastCol.plot(
            t/fs,
            significant_samples + int(stimLabel_to_sheet_order_map[stimLabel]),
            #color = 'yellow',
            label = stimLabel,
            #alpha = 0.3,
        )

        if 0: # overlay score images
            bar = 2 * (stimLabel_to_sheet_order_map[stimLabel] -1)
            # https://stackoverflow.com/questions/61169982/set-location-of-image-in-matplotlib
            image_artist = axLastCol.imshow(imgs[bar])

            if bar == 4: # 8 measures
                SCALE = 1803/fs #* 8/12.
            else: # 9 measures
                SCALE = 1803/fs #* 9/12.

            SCALE /= imgs[bar].shape[0]
            image_artist.set_extent(np.array(image_artist.get_extent())/SCALE)
            transform = mpl.transforms.Affine2D().translate(0, stimLabel_to_sheet_order_map[stimLabel])
            image_artist.set_transform(transform + axLastCol.transData)

        #ax_base.legend(loc='upper right', bbox_to_anchor=[1.3, 1])

    axLastCol.set_ylim([5, 0]) # flip axis to go top to bottom in song order of sheet music
    axLastCol.set_yticks(ticks=[1, 2, 3, 4], labels=['chor-038', 'chor-096', 'chor-101', 'chor-019'])
    #axLastCol.legend(loc='upper right')
    axLastCol.set_ylabel('Song #')
    axLastCol.set_xlabel('Time [s]')
    axLastCol.set_title(f'Subject{subjectID} | {condLabel} | Blink Avg > Null')

    pltIdx += 1 # since we already plotted the last column (bar codes)
    rowIdx += 1

plt.tight_layout()
        
plt.savefig(f'subs{len(allTrialsForNullDistro)}_{condLabel}_nullLoops{NUM_LOOPS}.png',
            format='png',
            dpi = 200,
            )

plt.show()



# Odd and Even Splits

In [ ]:
idxs

In [ ]:
NUM_LOOPS = 10000 #10000
NUM_TRIALS = 11
TEST_STIM_LABELS = ['chor-038', 'chor-096', 'chor-101', 'chor-019']
NOTES_Y_OFFSET = 1

PLOT_POINT_CLOUD = False # this takes a lot of memory => slow down

LOAD_DISTRO = not True
SAVE_DISTRO = not LOAD_DISTRO 

fs = 64

cond_trials = range(len(allTrialsForNullDistro[subjectID])) # this should be handled per subject if trials are dropped
split_trials_odd = [i for i in cond_trials if i%2==1]
split_trials_even = [i for i in cond_trials if i%2==0]

baseline = {} # this will be the null distro
test_trial_avg = {}
peaks_sq_baseline_distro = {}

t = np.arange(start=0, stop=len(cueExp), step=1)

plt.figure(figsize = (32, 16))

for split in ['even']: #, 'odd']:
    if split == 'odd':
        split_trials = split_trials_odd
    else:
        split_trials = split_trials_even
    
    pltIdx = 0
    rowIdx = 1

    for subjectID in allTrialsForNullDistro:
        
        if LOAD_DISTRO:
            peaks_sq_baseline_distro_samples = np.load(
                f"peaks_sq_baseline_distro_samples_sub{subjectID}_{','.join(condLabels)}_{NUM_LOOPS}_{split}.npy"
            )
            print(peaks_sq_baseline_distro_samples.shape)

        else:
            peaks_sq_baseline_distro[subjectID] = []

            # Draw NUM_TRIALS from all trials, and loop NUM_LOOPS times
            for n in range(NUM_LOOPS):
                peaks_sq_baseline = np.zeros_like(t, dtype=np.float64)
                
                randTrialIdxs = np.random.choice(
                    split_trials,
                    size=NUM_TRIALS,
                )
                #print(randTrialIdxs)

                for randn in randTrialIdxs: # decide which of 44 trials to draw
                    #print(randn)
                    peaks_sq_baseline += allTrialsForNullDistro[subjectID][randn]

                peaks_sq_baseline /= NUM_TRIALS # this gives us the average of 11 randomly selected trials
                peaks_sq_baseline_distro[subjectID].append(peaks_sq_baseline)

            #peaks_sq_baseline_distro_samples = np.empty_like(t) # this adds an extra row?
            i = 0
            for samples in peaks_sq_baseline_distro[subjectID]:
                if i == 0:
                    peaks_sq_baseline_distro_samples = samples
                else:
                    peaks_sq_baseline_distro_samples = np.vstack(
                        (peaks_sq_baseline_distro_samples, samples)
                    )
                i+=1
            print(peaks_sq_baseline_distro_samples.shape)

            # sort the samples for every time stamp to get the 95% interval
            peaks_sq_baseline_distro_samples.sort(axis=0)

            # Save to memory!
            if SAVE_DISTRO:
                np.save(
                    file=f"peaks_sq_baseline_distro_samples_sub{subjectID}_{','.join(condLabels)}_{NUM_LOOPS}_{split}",
                    arr=peaks_sq_baseline_distro_samples,
                )

        test_trial_avg[subjectID] = {}
        for stimLabel in TEST_STIM_LABELS:
            pltIdx += 1
            ax = plt.subplot(len(allTrialsForNullDistro), 4+1, pltIdx)

            # Plot null distro point cloud
            if PLOT_POINT_CLOUD:
                #for samples in peaks_sq_baseline_distro[subjectID]:
                for row in range(peaks_sq_baseline_distro_samples.shape[0]):
                    line, = ax.plot(
                        t/fs, 
                        peaks_sq_baseline_distro_samples[row], 
                        '.', 
                        #label='validation blinks', 
                        c='k',
                        alpha=0.1,
                        markersize=.1,
                    )
            else:
                max_line, = ax.plot(
                    t/fs, 
                    peaks_sq_baseline_distro_samples[-1, :], 
                    '.', 
                    #label='validation blinks', 
                    c='k',
                    alpha=0.1,
                    #markersize=.1,
                    label = 'Max[null]'
                )

            # Plot null distro CI intervals
            null_distro_95 = peaks_sq_baseline_distro_samples[int(.95*NUM_LOOPS), :]

            null_line, = ax.plot(
                t/fs, 
                null_distro_95, 
                #marker = '_',
                #ls = '', 
                #label='validation blinks', 
                c='c',
                #alpha=0.1,
            )

            ax.fill_between(
                t/fs, 
                peaks_sq_baseline_distro_samples[int(.95*NUM_LOOPS), :], 
                0, 
                color=null_line.get_color(), 
                alpha=.3,
                label = 'Null Distribution'
            )

            test_trial_avg[subjectID][stimLabel] = np.zeros_like(t, dtype=np.float64)

            notes = t[stim[stimLabel]['notes']]
            beats = t[stim[stimLabel]['beats']>0]
            
            for condLabel in condLabels:
                Navg = 0
                for trialIdx in peaks_sq_all[subjectID][stimLabel][condLabel]:
                    if trialIdx in split_trials:
                        Navg += 1
                        val_peaks_sq = peaks_sq_all[subjectID][stimLabel][condLabel][trialIdx]
                        test_trial_avg[subjectID][stimLabel] += val_peaks_sq
                    
                test_trial_avg[subjectID][stimLabel] /= Navg #len(peaks_sq_all[subjectID][stimLabel][condLabel])

                test_distro = test_trial_avg[subjectID][stimLabel]

                avg_line, = ax.plot(
                    t/fs, 
                    test_distro, 
                    #'.', 
                    #label=f'Trial avg. for {stimLabel}', 
                    c='r',
                    #alpha=0.1,
                )

                ax.fill_between(
                    t/fs, 
                    test_distro, 
                    0, 
                    color=avg_line.get_color(), 
                    alpha=.3,
                    label=f'Trial avg. for {stimLabel}', 
                )

                significant_samples = (test_distro - null_distro_95) > 0

                sig_line, = ax.plot(
                    t/fs,
                    significant_samples,
                    color = 'yellow',
                    alpha = 0.3,
                )
                ax.fill_between(
                    t/fs, 
                    significant_samples, 
                    0, 
                    color=sig_line.get_color(), 
                    label = 'Test > 95%ile of null',
                    alpha=.3,
                )

                score = 100 * np.sum(significant_samples) / len(significant_samples)

            ax.plot(t[notes]/fs, 0*t[notes] + NOTES_Y_OFFSET, '.', label='notes')
            beat_dots, = ax.plot(t[beats]/fs, 0*t[beats] + NOTES_Y_OFFSET, 'o', label='beats')
            for beat in beats:
                ax.axvline(
                    x=t[beat]/fs, 
                    ls='-', # solid line
                    alpha=0.2, 
                    c=beat_dots.get_color()
                )

            #ax_base = ax.twinx()
            #ax_base.plot(t/fs, peaks_sq_baseline, label='baseline') #, c=line.get_color())
            
            title_str = f'Subject{subjectID} | {condLabel} | {stimLabel} vs. Null'
            title_str+= f'\n#Trials Per Avg: {NUM_TRIALS} | #Loops: {NUM_LOOPS} | {split}'
            title_str+= '\nTest > Null for {:.2f}% time'.format(score)
            ax.set_title(title_str)

            ax.set_xlabel('Time [s]')
            ax.set_ylim([-.1, 1.1])

            if (pltIdx % 5) == 1: # first column
                ax.legend(loc='upper right', bbox_to_anchor=[-0.1, 1])

            # Plot the significance curves for all songs in the last column
            axLastCol = plt.subplot(
                len(allTrialsForNullDistro), # nrows = no. of subjects
                4+1, # cols = no. of songs + one "bar codes" plot
                5*rowIdx, # select the last column of this row
            )
            sig_line, = axLastCol.plot(
                t/fs,
                significant_samples + int(stimLabel_to_sheet_order_map[stimLabel]),
                #color = 'yellow',
                label = stimLabel,
                #alpha = 0.3,
            )

            if 0: # overlay score images
                bar = 2 * (stimLabel_to_sheet_order_map[stimLabel] -1)
                # https://stackoverflow.com/questions/61169982/set-location-of-image-in-matplotlib
                image_artist = axLastCol.imshow(imgs[bar])

                if bar == 4: # 8 measures
                    SCALE = 1803/fs #* 8/12.
                else: # 9 measures
                    SCALE = 1803/fs #* 9/12.

                SCALE /= imgs[bar].shape[0]
                image_artist.set_extent(np.array(image_artist.get_extent())/SCALE)
                transform = mpl.transforms.Affine2D().translate(0, stimLabel_to_sheet_order_map[stimLabel])
                image_artist.set_transform(transform + axLastCol.transData)

            #ax_base.legend(loc='upper right', bbox_to_anchor=[1.3, 1])

        axLastCol.set_ylim([5, 0]) # flip axis to go top to bottom in song order of sheet music
        axLastCol.set_yticks(ticks=[1, 2, 3, 4], labels=['chor-038', 'chor-096', 'chor-101', 'chor-019'])
        #axLastCol.legend(loc='upper right')
        axLastCol.set_ylabel('Song #')
        axLastCol.set_xlabel('Time [s]')
        axLastCol.set_title(f'Subject{subjectID} | {condLabel} | Blink Avg > Null')

        pltIdx += 1 # since we already plotted the last column (bar codes)
        rowIdx += 1

    plt.tight_layout()
            
    plt.savefig(f'subs{len(allTrialsForNullDistro)}_{condLabel}_nullLoops{NUM_LOOPS}_{split}.png',
                format='png',
                dpi = 200,
                )

plt.show()



In [ ]:
null_distro_95 = peaks_sq_baseline_distro_samples[int(.95*NUM_LOOPS), :]
test_distro = test_trial_avg[subjectID][stimLabel]

plt.figure(figsize = (9,4))
plt.plot(null_distro_95)
plt.plot(test_distro, ':')
plt.plot((test_distro - null_distro_95) > 0)
plt.show()

In [ ]:
test_trial_avg[subjectID][stimLabel]